In [94]:
import json
import re
import mistune

In [101]:
def parse_options(options):
    options_dict = []
    for option in options:
        label = option[0]
        content = option[2:]
        option_dict = {
            'label': label,
            'content': content
        }
        options_dict.append(option_dict)
    return options_dict

def wrap_latex_with_span(text):
    patterns = [r'\\\((.*?)\\\)', r'\\\[(.*?)\\\]', r'\$\$(.*?)\$\$']
    for i in range(3):
        if i == 0:
            pattern = patterns[i]
            def replace_latex(match):
                return r'<span class="math-tex">\({}\)</span>'.format(match.group(1))
            text = re.sub(pattern, replace_latex, text, flags=re.DOTALL)
        if i == 1:
            pattern = patterns[i]
            def replace_latex(match):
                return r'<span class="math-tex">\[{}\]</span>'.format(match.group(1))
            text = re.sub(pattern, replace_latex, text, flags=re.DOTALL)
        if i == 2:
            pattern = patterns[i]
            def replace_latex(match):
                return r'<span class="math-tex">\[{}\]</span>'.format(match.group(1))
            text = re.sub(pattern, replace_latex, text, flags=re.DOTALL)
    return text

def support_render_on_mobile(text_data):
    
    if text_data:
        text_data = text_data.replace('&amp;', '&').replace('&nbsp;', ' ').replace('&lt;', '<').replace('&gt;', '>').replace('&quot;', '"').replace('&apos;', "'").replace('operatorname', 'text')

        pattern = r'<span class="math-tex">\\\[.*?\\\]</span>'
        matches = re.findall(pattern, text_data)

        for match in matches:
            new_match = match.replace('\\[', '\\(').replace('\\]', '\\)')
            text_data = text_data.replace(match, new_match)

        pattern = r'\\overparen\{.*?\}'
        matches = re.findall(pattern, text_data)

        for match in matches:
            content = match[11:-1]  # 11 is the length of '\overparen{', -1 is to exclude the closing brace
            new_match = '\\text{cung } ' + content
            text_data = text_data.replace(match, new_match)
        
        return text_data
    return text_data

def escape_markdown_special_characters(text):
    escape_chars = {
        # '\\': '\\\\',
        '`': '\\`',
        '*': '\\*',
        '_': '\\_',
        # '{': '\\{',
        # '}': '\\}',
        # '[': '\\[',
        # ']': '\\]',
        # '(': '\\(',
        # ')': '\\)',
        '#': '\\#',
        '+': '\\+',
        '-': '\\-',
        '.': '\\.',
        '!': '\\!',
    }
    return text.translate(str.maketrans(escape_chars))

def transform_data(text_data):
    if text_data:
        html_parser = mistune.create_markdown(
            escape=False,
            plugins=['strikethrough', 'footnotes', 'table', 'speedup', 'url', 'math', 'superscript', 'subscript']
        )
        # text_data = escape_markdown_special_characters(text_data)
        text_data = wrap_latex_with_span(text_data)
        text_data = "<p>" + text_data + "</p>"
        text_data = support_render_on_mobile(text_data)
        return text_data
    return text_data

In [102]:
data = json.load(open('/home/dica/Projects/syvan_projects/pdf_parser/pdf_parser/paper_exams/003/exam_questions_de-giua-ky-1-toan-12-nam-2022-2023-truong-thpt-phu-cu-hung-yen-HanhTruong.json', 'r'))

In [103]:
data[0]

{'question_title': 'Câu 1',
 'original_text': 'Giá trị nhỏ nhất của hàm số \\(y = x^4 -3x^2\\) trên \\([-1;1]\\) là:',
 'quiz_options': ['A. - 2.', 'B. 0.', 'C. - 4.', 'D. - 5.'],
 'correct_option': 'C',
 'question_type': 'multiple-choice',
 'explanation': ''}

In [104]:
options = parse_options(data[0]['quiz_options'])
options

[{'label': 'A', 'content': ' - 2.'},
 {'label': 'B', 'content': ' 0.'},
 {'label': 'C', 'content': ' - 4.'},
 {'label': 'D', 'content': ' - 5.'}]

In [105]:
transform_data(options[0]['content'])

'<p> - 2.</p>'

In [106]:
print(transform_data(data[0]['original_text']))

<p>Giá trị nhỏ nhất của hàm số <span class="math-tex">\(y = x^4 -3x^2\)</span> trên <span class="math-tex">\([-1;1]\)</span> là:</p>


In [70]:
transform_data(data[0]['original_text'])

'<p>Giá trị nhỏ nhất của hàm số <span class="math-tex">\\(y = x^4 -3x^2\\)</span> trên <span class="math-tex">\\([-1;1]\\)</span> là:</p>\n'

In [71]:
transform_data("()")

'<p>()</p>\n'

In [80]:
import json
prompt_collection = json.load(open("/home/dica/Projects/syvan_projects/pdf_parser/pdf_parser/src/prompt_collections/collection_01.json", "r"))

In [82]:
prompt_collection[0]

{'extract_information': 'Trích xuất toàn bộ thông tin của đề thi ở trang đầu tiên? Trả lời dưới dạng JSON chính xác và đẩy đủ thông tin. Danh sách keys:\n- exam_code (chỉ trả lời ở dạng số),\n- number_of_questions (chỉ trả lời ở dạng số),\n- school,\n- subject,\n- grade,\n- duration (chỉ trả lời ở dạng số),\n- year (trả lời đẩy đủ năm học),\n- exam_term (chọn một trong danh sách sau: [Giữa kỳ I, Cuối kỳ I, Giữa kỳ II, Cuối kỳ II, Thi vào lớp 10, Thi THPT, Thi học sinh giỏi, Chất lượng đầu năm, Đánh giá năng lực, Ôn tập chương]),\n- exam_full_name (trả lời đẩy đủ và chính xác tên kỳ thi như trong tài liệu)',
 'get_correct_answer_table': ['Cung cấp bảng chứa đáp án cho từng mã đề dưới dạng bảng, chính xác và đầy đủ thông tin.',
  'Dựa vào bảng trên và mã đề, hãy cung cấp đáp án đúng cho từng câu hỏi trong đề dưới dạng bảng thật chính xác.'],
 'format_and_function_tag': '- Đối với câu hỏi trắc nghiệm thì định dạng sẽ là:\n{\n        "question_title": "Câu 1",\n        "original_text": "Cá

In [87]:
print(prompt_collection[0]['format_and_function_tag'])

- Đối với câu hỏi trắc nghiệm thì định dạng sẽ là:
{
        "question_title": "Câu 1",
        "original_text": "Cách viết nào sau đây cho ta một phân số?",
        "quiz_options": "A. \\(\\frac{2,5}{7}\\). B. \\(\\frac{8}{0}\\). C. \\(\\frac{-2}{11}\\). D. \\(\\frac{-3}{3,7}\\)",
        "correct_option": "C",
        "question_type": "multiple-choice",
        "explanation": ""
}
- Đối với câu hỏi tự luận thì định dạng sẽ là:
{
        "question_title": "Câu 1",
        "original_text": "Có bao nhiêu giá trị nguyên của m?",
        "question_type": "essay",
        "explanation": ""  
}
- Function tag:
/extract(X): trích xuất nội dung câu hỏi X theo định mẫu bên trên, với công thức toán thì sử dụng LaTeX trong dấu phân cách \(...\) chắc chắn hiển thị được, hình ảnh bao gồm bảng biến thiên, hình minh hoạ, đồ thị,... có thể bỏ qua, và đảm bảo trích xuất văn bản một cách kỹ lưỡng và chính xác từ tệp PDF được cung cấp, đáp án lấy từ JSON phía trên. Ví dụ: /extract(1)

Trả lời "OK" nếu b

In [89]:
options = ["A. \\(\\frac{2,5}{7}\\).", "B. \\(\\frac{8}{0}\\).", "C. \\(\\frac{-2}{11}\\).", "D. \\(\\frac{-3}{3,7}\\)"]

In [93]:
options[0][:2], options[0][2:]

('A.', ' \\(\\frac{2,5}{7}\\).')

In [114]:
# a = "hello \nworld"
a = "hello world"
print(a)

hello world


In [115]:
b = "".join("<p>" + chunk + "</p>" for chunk in a.split("\n"))
b

'<p>hello world</p>'

In [116]:
for chunk in a.split("\n"):
    print(chunk)

hello world
